# Plan

- 







In [76]:
# imports


from google.colab import files
import pandas as pd
import io
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import numpy as np
import string
from keras.utils import to_categorical
from bpe import Encoder
from collections import defaultdict


In [ ]:
!pip install bpe

In [3]:
# load data

uploaded = files.upload()

Saving babynames-clean.csv to babynames-clean.csv


In [71]:
names = pd.read_csv(io.BytesIO(uploaded['babynames-clean.csv']))
names.drop(names.columns[[1, 1]], axis=1, inplace=True)

# print(names)






In [ ]:
##########################
#         BPE            #
##########################

corpus = names["John"].tolist() # to list ify

encoder = Encoder(800, pct_bpe=1)  # encode it all
encoder.fit(corpus) # fit it to the corpus

# example = "William"

names["tokenized_names"] = names.apply(lambda row:encoder.tokenize(row["John"]), axis=1) # amke the collumn
# print(encoder.tokenize(example))
# print(names.head())
# names.tail()
print(names["tokenized_names"])
# print(corpus)


In [73]:
## find max_len


high = 0
high = names.apply(lambda row: max(high, len(row["tokenized_names"])), axis=1)
max_len = max(high.tolist())



In [ ]:
##########################
#       Padding          #
##########################

names["padded"] = names.apply(lambda row: row["tokenized_names"]+(max_len - len(row["tokenized_names"]))*["__pad"], axis=1) 

names.head()



In [110]:
##########################
#       Tokenize         #
##########################

tok = defaultdict(lambda: len(tok))

# names["toked"] = names.apply(lambda row: lambda item: tok[item], axis=1)

names["toked"] = [[tok[j] for j in i] for i in names["padded"].tolist()]

# up = 0
# for i in names["toked"].tolist():
#   for j in i:
#     up = max(up, j)
# print(up)

# print(len(tok))
names.head()



,John,tokenized_names,padded,toked,one_hot
0,William,"[__sow, wi, ll, ia, m, __eow]","[__sow, wi, ll, ia, m, __eow, __pad, __pad]","[0, 1, 2, 3, 4, 5, 6, 6]",None
1,James,"[__sow, ja, me, s, __eow]","[__sow, ja, me, s, __eow, __pad, __pad, __pad]","[0, 7, 8, 9, 5, 6, 6, 6]",None
2,Charles,"[__sow, ch, ar, le, s, __eow]","[__sow, ch, ar, le, s, __eow, __pad, __pad]","[0, 10, 11, 12, 9, 5, 6, 6]",None
3,George,"[__sow, ge, or, ge, __eow]","[__sow, ge, or, ge, __eow, __pad, __pad, __pad]","[0, 13, 14, 13, 5, 6, 6, 6]",None
4,Frank,"[__sow, fr, an, k, __eow]","[__sow, fr, an, k, __eow, __pad, __pad, __pad]","[0, 15, 16, 17, 5, 6, 6, 6]",None


In [120]:
##########################
#       One Hot          #
##########################

names["one_hot"] = names.apply(lambda row: to_categorical(row["toked"], num_classes=434), axis=1)
# names["one_hot"] = names.apply(lambda row: print(row["toked"]), axis=1)


# ex = [0, 1, 2, 3, 4, 5, 6, 6]

# print(to_categorical(ex, num_classes=800))
# print(names["one_hot"])
# print(names.apply(lambda row: row["one_hot"], axis=1))

# for i in names["one_hot"].tolist()[1:2]:

  # print(i)
# names.head()

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:

# t = Tokenizer(char_level = True)
# # fit the tokenizer on the documents
# t.fit_on_texts(names)

# print(t)
# # summarize what was learned
# # print(t.word_counts)
# # print(t.document_count)
# # print(t.word_index)
# # print(t.word_docs)
# # # integer encode documents
# # encoded_docs = t.texts_to_matrix(names, mode='count')
# # print(encoded_docs)



# # sequence_of_int = t.texts_to_sequences(names)
# # train = tf.keras.utils.to_categorical(sequence_of_int)
# # print(sequence_of_int)
# # print(train)
# # print(names)


In [42]:
# alphabets = {'a' : 0, 'b': 1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7, 'i':8, 'j':9, 'k':10, 'l':11, 'm':12, 'n':13, 'o':14, 'p':15, 'q':16, 'r':17, 's':18, 't':19, 'u':20, 'v':21, 'w':22, 'x':23, 'y':24, 'z':25}


# idxs = [alphabets[ch] for ch in 'hello']
# print(idxs)
# [7, 4, 11, 11, 14]

# @divakar's approach
# idxs = np.fromstring("hello",dtype=np.uint8)-97

# or for more clear understanding, use:

def vec(strng, alphabet=string.ascii_lowercase):
    vector = [[0 if char != letter else 1 for char in alphabet] 
                  for letter in strng]
    return vector

# train = []

# for name in names[:5]:
  # idxs = np.fromstring(name, dtype=np.uint8) - ord('a')
  # one_hot = tf.one_hot(idxs, 26, dtype=tf.uint8)
  # train.append(one_hot)
  # train.append(vec(name))
  # print(name, vec(name))

# names.applymap(vec)
# names.applymap(lambda x: len(str(x)))

print(names)



          John
0      William
1        James
2      Charles
3       George
4        Frank
...        ...
6776    Laylah
6777  Carleigh
6778    Kenley
6779    Sloane
6780   Elianna

[6781 rows x 1 columns]
